In [1]:
from dhanhq import dhanhq, marketfeed
import talib
import numpy as np
import pandas as pd
import time
import datetime
import os
import logging

In [2]:
with open("token.txt", "r") as file:
    # Read the entire contents of the file
    token = file.read()
client_id = "1104488151"
access_token = token
dhan = dhanhq(client_id,access_token)

masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', nrows=1)
masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', usecols=masterlist.columns, low_memory=False)


logging.basicConfig(filename="logfile.txt", 
                    level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


In [4]:
NIFTY_PREMIMUM = 23550

filtered_ce_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-02-13 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'CE') 
]
filtered_ce_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
141822,NSE,D,51219,OPTIDX,0,NIFTY-Feb2025-23550-CE,75.0,NIFTY 13 FEB 23550 CALL,2025-02-13 14:30:00,23550.0,CE,5.0,W,OP,NaN,NaN


In [5]:
filtered_pe_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-02-13 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'PE')
]

filtered_pe_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
141823,NSE,D,51220,OPTIDX,0,NIFTY-Feb2025-23550-PE,75.0,NIFTY 13 FEB 23550 PUT,2025-02-13 14:30:00,23550.0,PE,5.0,W,OP,NaN,NaN


In [ ]:

SECURITY_ID = 51220

IS_ACTIVE = False
IS_OPEN_POSITION = False
quantity = 0
buy_price = 0  
stoploss_point = 2   
stop_loss = 0
target_point = 1
target = 0

while IS_ACTIVE is False:
        current_second = datetime.datetime.now().second
        time.sleep(1)
        print("waiting to activate", current_second)
        if current_second == 1:
            IS_ACTIVE = True

while IS_ACTIVE is True and IS_OPEN_POSITION is False:
      current_second = datetime.datetime.now().second
      if current_second == 0:
            break
      print('Active',current_second)
      time.sleep(0.15)
      intraday = dhan.intraday_daily_minute_charts(SECURITY_ID, dhan.NSE_FNO, "OPTIDX")
      print("intraday", intraday)
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

      for i in intraday_df['start_Time']:
          temp = dhan.convert_to_date_time(i)
          temp_list.append(temp)

      intraday_df['Date'] = temp_list

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]
      
      # buy_price = int(current_running_candle['close'])
    #   stop_loss = int(previous_closed_candle['low'])
      buy_price = current_running_candle['close']

      quantity                  = 150
      target                    = buy_price + target_point
      stop_loss                 = buy_price - stoploss_point

      dhan.place_order(
      security_id=SECURITY_ID,
      exchange_segment=dhan.NSE_FNO,
      transaction_type=dhan.BUY,
      quantity=quantity,
      order_type=dhan.MARKET,
      product_type=dhan.INTRA,
      price=0)  

      logging.info(f"BUY at {buy_price}")

      IS_OPEN_POSITION = True
      break




waiting to activate 0
waiting to activate 1
Active 2
intraday {'status': 'success', 'remarks': '', 'data': {'open': [39.65, 35.8, 29.25, 30.0, 28.1, 32.55, 34.35, 38.35, 50.0, 44.7, 47.9, 55.45, 52.35, 54.1, 55.35, 53.35, 57.75, 61.75, 66.0, 74.2, 78.65, 73.65, 74.85, 75.5, 70.45, 76.7, 73.65, 75.5, 69.0, 64.1, 64.15, 56.7, 59.65, 63.15, 61.9, 66.5, 67.65, 75.2, 71.9, 67.65, 64.45, 70.2, 72.1, 78.2, 70.5, 66.65, 68.35, 70.25, 71.75, 64.35, 56.4, 57.0, 57.1, 59.0, 64.75, 72.45, 66.85, 69.15, 74.15, 70.9, 73.2, 68.6, 69.2, 65.85, 67.6, 62.75, 58.25, 62.65, 62.7, 59.7, 53.6, 55.65, 55.6, 60.4, 63.4, 64.25, 58.65, 55.2, 57.95, 62.5, 58.7, 58.05, 53.3, 50.7, 46.55, 44.75, 41.5, 45.35, 47.4, 44.15, 46.0, 44.25, 46.1, 48.8, 51.4, 54.65, 46.1, 44.35, 46.8, 48.2, 51.85, 51.25, 48.45, 47.05, 45.0, 45.75, 44.9, 51.25, 53.4, 53.45, 54.35, 69.15, 66.85, 67.8, 64.3, 66.35, 66.05, 67.4, 73.95, 69.45, 67.15, 70.5, 72.05, 69.65], 'high': [41.25, 35.8, 31.15, 30.0, 34.85, 34.8, 42.0, 47.8, 56.2, 49.4, 6

In [28]:

COST_TO_COST_SL = False
TRAILING_SL = False

while IS_OPEN_POSITION is True and quantity > 0:
      time.sleep(0.10)
      intraday = dhan.intraday_daily_minute_charts(SECURITY_ID, dhan.NSE_FNO, "OPTIDX")
     
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

      for i in intraday_df['start_Time']:
          temp = dhan.convert_to_date_time(i)
          temp_list.append(temp)

      intraday_df['Date'] = temp_list

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]

      print("\n" + "*" * 45)  # Adds a separator

      print(f"{'LTP':<30}: {current_running_candle['close']}")
      print(f"{'Stop Loss':<30}: {stop_loss}")
      print(f"{'Buy Price':<30}: {buy_price}")
      print(f"{'POINTS':<30}: {current_running_candle['close'] - buy_price}")

      print("\n" + "*" * 45)  # Adds a separator



      #  # Quick Scalping target
      # if current_running_candle['close'] > target and COST_TO_COST_SL is False and TRAILING_SL is False:
      #       print("SELL")
      #       IS_OPEN_POSITION = False
          
      #      # Place an order for NSE Futures & Options
      #       dhan.place_order(security_id=SECURITY_ID,
      #       exchange_segment=dhan.NSE_FNO,
      #       transaction_type=dhan.SELL,
      #       quantity=quantity,
      #       order_type=dhan.MARKET,
      #       product_type=dhan.INTRA,
      #       price=0)  
      #       logging.info(f"Target hit {current_running_candle['close']}")

      # Stop loss
      if current_running_candle['close'] < stop_loss and COST_TO_COST_SL is False and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  
            logging.info(f"SL hit {current_running_candle['close']}")

      # Trailing SL
      if current_running_candle['close'] < previous_closed_candle['low'] and COST_TO_COST_SL is False and TRAILING_SL is True:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  
            logging.info(f"Trailing SL hit {current_running_candle['close']}")

      # Shift sl to Cost to Cost
      if current_running_candle['close'] < buy_price +1 and COST_TO_COST_SL is True and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  
            logging.info(f"CTC SL hit {current_running_candle['close']}")

            

waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.8
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  70.1
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.9
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.9
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.9
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.9
Stop Loss set at Price :-  67
waiting for sell, LTP is :-  69.9
Stop L

KeyboardInterrupt: 